In [1]:
# BackBone 모델 : MobileNetV2 활용
# 희귀한 소량의 이미지 데이터를 CIFAR-10 데이터로 가정
# 실습 2 : MobileNetV2 모델 학습시켜 내 이미지 데이터 모델을 잘 분류하는 모델 생성
# Transfer Learning, Fine Tunning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# 데이터 불러오기
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print(x_train.shape)
num_classes = 10

# 데이터 스케일링
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('train data :', x_train.shape, y_train.shape)
print('test data :', x_test.shape, y_test.shape)

(50000, 32, 32, 3)
train data : (50000, 32, 32, 3) (50000, 10)
test data : (10000, 32, 32, 3) (10000, 10)


In [4]:
# Transfer Learning (전이학습) : 기본 모델의 가중치는 모두 동결 freeze -> 새로 추가한 분류층만 학습
base_model = keras.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,    # 분류기 부분을 빼고 convolution만 가지고 모델 생성
    weights='imagenet',    # 사전 학습된 가중치를 그대로 사용한다
)

base_model.trainable = False   # 동결 ; 분류기만 쓰겠다. convolution(원래 꺼 사용) 안쓰겠다.

# 함수형 api로 모델 생성
inputs = keras.Input(shape=(32,32,3))
x = layers.Resizing(96, 96)(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)   # MaxPooling보다 더 급격하게 feature의 크기를 줄인다.
outputs = layers.Dense(num_classes, activation='softmax')(x)
model_t1 = keras.Model(inputs, outputs)

# 학습
model_t1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_t1.fit(x_train, y_train, epochs=3, batch_size=32, validation_split=0.1, verbose=1)

# 평가
print(f"test 평가 결과 : {model_t1.evaluate(x_test, y_test, verbose=0)}")

2025-09-24 12:05:14.054812: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-24 12:05:14.055949: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-24 12:05:14.055954: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-24 12:05:14.056003: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-24 12:05:14.056182: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3


2025-09-24 12:05:18.152044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1407/1407 ━━━━━━━━━━━━━━━━━━━━ 60s 40ms/step - accuracy: 0.7434 - loss: 0.7419 - val_accuracy: 0.7886 - val_loss: 0.6001
Epoch 2/3
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - accuracy: 0.7975 - loss: 0.5816 - val_accuracy: 0.7968 - val_loss: 0.5818
Epoch 3/3
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - accuracy: 0.8123 - loss: 0.5377 - val_accuracy: 0.8006 - val_loss: 0.5671
test 평가 결과 : [0.5844452381134033, 0.8022000193595886]


In [5]:
# Fine-Tunning (미세조정)
# 베이스 모델 일부 층만 열기(예 : 마지막 30개)

for layer in base_model.layers[-30:]:   # 마지막 30개만 trainable=True
    layer.trainable = True
'''
base_model.trainable = True
for layer in base_model.layers[:-30]:   # 마지막 30개를 제외하고 나머지 동결
    layer.trainable = False
'''

# 이제 낮은 학습률(learning rate)으로 다시 컴파일
# weight를 너무 변화를 주면 안된다.(성능이 잘 나오기 때문이다.)
model_t1 = keras.Model(inputs, outputs)

# 학습
model_t1.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history = model_t1.fit(x_train, y_train, epochs=3, batch_size=32, validation_split=0.1, verbose=1)

# 평가
print(f"test 평가 결과 : {model_t1.evaluate(x_test, y_test, verbose=0)}")

Epoch 1/3
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 137s 89ms/step - accuracy: 0.7367 - loss: 0.8502 - val_accuracy: 0.8020 - val_loss: 0.6455
Epoch 2/3
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 129s 91ms/step - accuracy: 0.8051 - loss: 0.5947 - val_accuracy: 0.8118 - val_loss: 0.5860
Epoch 3/3
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 115s 81ms/step - accuracy: 0.8314 - loss: 0.4944 - val_accuracy: 0.8216 - val_loss: 0.5480
test 평가 결과 : [0.5670218467712402, 0.8198999762535095]
